In [1]:
import pandas as pd
import os

In [2]:
pd.set_option('display.max_colwidth', None)

Estimated expression levels from RSEM as a tsv file. The columns are as follows:

column 1: gene_id - gene name of the gene the transcript belongs to (parent gene). If no gene information is provided, gene_id and transcript_id is the same.  
column 2: transcript_id(s) - transcript name of this transcript  
column 3: length - the transcript's sequence length (poly(A) tail is not counted)  
column 4: effective_length - the length containing only the positions that can generate a valid fragment  
column 5: expected_count - the sum of the posterior probability of each read comes from this transcript over all reads  
column 6: TPM - transcripts per million, a measure of relative measure of transcript abundance  
column 7: FPKM - fragments per kilobase of transcript per million mapped reads, another relative measure of transcript abundance  
column 8: posterior_mean_count - posterior mean estimate calcualted by RSEM's Gibbs sampler  
column 9: posterior_standard_deviation_of_count - posterior standard deviation of counts  
column 10: pme_TPM - posterior mean estimate of TPM  
column 11: pme_FPKM - posterior mean estimate of FPKM  
column 12: TPM_ci_lower_bound - lower bound of 95% credibility interval for TPM values  
column 13: TPM_ci_upper_bound - upper bound of 95% credibility interval for TPM values  
column 14: FPKM_ci_lower_bound - lower bound of 95% credibility interval for FPKM values  
column 15: FPKM_ci_upper_bound - upper bound of 95% credibility interval for FPKM values  

In [3]:
# load Mouse ENCODE E11.5 data
limb_1 = pd.read_csv("../BEHST/data/ENCFF195JHC.tsv", sep='\t', header=0)
limb_2 = pd.read_csv("../BEHST/data/ENCFF457ZGF.tsv", sep='\t',header=0)
forebrain_1 = pd.read_csv("../BEHST/data/ENCFF465SNB.tsv", sep='\t', header=0)
forebrain_2 = pd.read_csv("../BEHST/data/ENCFF976OLT.tsv", sep='\t', header=0)
midbrain_1 = pd.read_csv("../BEHST/data/ENCFF359ZOA.tsv", sep='\t', header=0)
midbrain_2 = pd.read_csv("../BEHST/data/ENCFF971KZC.tsv", sep='\t', header=0)
hindbrain_1 = pd.read_csv("../BEHST/data/ENCFF750FTK.tsv", sep='\t', header=0)
hindbrain_2 = pd.read_csv("../BEHST/data/ENCFF109HTF.tsv", sep='\t', header=0)
heart_1 = pd.read_csv("../BEHST/data/ENCFF226IWR.tsv", sep='\t', header=0)
heart_2 = pd.read_csv("../BEHST/data/ENCFF540EJL.tsv", sep='\t', header=0)
liver_1 = pd.read_csv("../BEHST/data/ENCFF954EHG.tsv", sep='\t', header=0)
liver_2 = pd.read_csv("../BEHST/data/ENCFF523MEO.tsv", sep='\t', header=0)
neural_tube_1 = pd.read_csv("../BEHST/data/ENCFF375JDR.tsv", sep='\t', header=0)
neural_tube_2 = pd.read_csv("../BEHST/data/ENCFF298WHK.tsv", sep='\t', header=0)
facial_1 = pd.read_csv("../BEHST/data/ENCFF772UWT.tsv", sep='\t', header=0)
facial_2 = pd.read_csv("../BEHST/data/ENCFF262TXH.tsv", sep='\t', header=0)

In [4]:
# Ensemble orthologous genes
gene_ids = pd.read_csv("../BEHST/data/human_mouse_gene_id.txt", sep='\t')
gene_ids = gene_ids.rename(columns={'Mouse gene stable ID': 'gene_id'})

In [5]:
gene_ids.head()

,Gene stable ID,gene_id,Mouse gene name,Mouse homology type
0,ENSG00000198888,ENSMUSG00000064341,mt-Nd1,ortholog_one2one
1,ENSG00000198763,ENSMUSG00000064345,mt-Nd2,ortholog_one2one
2,ENSG00000198804,ENSMUSG00000064351,mt-Co1,ortholog_one2one
3,ENSG00000198712,ENSMUSG00000064354,mt-Co2,ortholog_one2one
4,ENSG00000228253,ENSMUSG00000064356,mt-Atp8,ortholog_one2one


#### Defining Tissue-specific Genes
Tissue-specific genes are defined using the algorithm from the HPA (Uhlén et al. 2015), and can be grouped as follows:

- Tissue Enriched: Genes with an expression level greater than 1 (TPM or FPKM) that also have at least five-fold higher expression levels in a particular tissue compared to all other tissues.
- Group Enriched: Genes with an expression level greater than 1 (TPM or FPKM) that also have at least five-fold higher expression levels in a group of 2-7 tissues compared to all other tissues, and that are not considered Tissue Enriched.
- Tissue Enhanced: Genes with an expression level greater than 1 (TPM or FPKM) that also have at least five-fold higher expression levels in a particular tissue compared to the average levels in all other tissues, and that are not considered Tissue Enriched or Group Enriched.



In [6]:
def calculate_mean(df_list):
    """
    Calculate the mean values of the isogenic replicates of gene quantifications.
    """
    df_all = pd.concat(df_list)
    df_mean = df_all.groupby(['gene_id']).mean()
    df_mean = df_mean.reset_index()
    
    # TODO: can drop unused columns here
    return df_mean

In [7]:
def select_genes(tissue_dfs, other_dfs):
    """
    select most highly expressed genes in this tissue
    """
    tissue_mean = calculate_mean(tissue_dfs)
    other_mean = calculate_mean(other_dfs)
    
    # select highly expressed genes in this tissue
    tissue_genes = tissue_mean[tissue_mean['pme_TPM'] > 1]
    
    # compare with other types
    tissue_enhanced = pd.merge(tissue_genes, other_mean, on='gene_id')
    tissue_enhanced = tissue_enhanced.rename(columns={'pme_TPM_x': 'tissue_TPM', 'pme_TPM_y': 'other_TPM'})
    # select useful columns
    tissue_enhanced = tissue_enhanced[['gene_id', 'tissue_TPM', 'other_TPM']]
    
    df_enhanced = tissue_enhanced[tissue_enhanced['tissue_TPM'] >= (5 * tissue_enhanced['other_TPM'])].copy()
    
    # convert to mouse stable id
    df_enhanced['gene_id'] = df_enhanced['gene_id'].str.split('.').str.get(0)
    
    return df_enhanced

## BEHST Limb

In [8]:
# select most highly expressed genes in limb
limb_df = select_genes([limb_1, limb_2], 
                       [forebrain_1, forebrain_2, midbrain_1, midbrain_2, hindbrain_1, hindbrain_2, 
                       heart_1, heart_2, liver_1, liver_2, neural_tube_1, neural_tube_2, facial_1, facial_2])

len(limb_df)

97

In [9]:
# select genes with mappable id for homo sapiens
mapped = pd.merge(limb_df, gene_ids, on='gene_id', how='inner')
len(mapped)

# mapped[['Gene stable ID']].to_csv('../BEHST/data/06_24_reference_limb_gene_id', header=None, index=None)

69

In [10]:
# run gprofiler2 on the mapped genes, get this GO list as reference
limb_ref = pd.read_csv("../BEHST/results/06-24/limb_go_list", sep="\t", header=0)
len(limb_ref)

2336

In [11]:
# select significant terms
limb_ref = limb_ref[limb_ref.p_value <= 0.05]
limb_ref

,p_value,term_id,source,term_name
0,1.520378e-26,GO:0060173,GO:BP,limb development
1,1.520378e-26,GO:0048736,GO:BP,appendage development
2,5.502065e-23,GO:0035107,GO:BP,appendage morphogenesis
3,5.502065e-23,GO:0035108,GO:BP,limb morphogenesis
4,1.153630e-22,GO:0035113,GO:BP,embryonic appendage morphogenesis
...,...,...,...,...
173,3.137960e-02,GO:0044271,GO:BP,cellular nitrogen compound biosynthetic process
174,3.174007e-02,GO:0045445,GO:BP,myoblast differentiation
175,3.226027e-02,GO:0072006,GO:BP,nephron development
176,3.943211e-02,GO:0003676,GO:MF,nucleic acid binding


In [12]:
# read BEHST limb output GO terms
limb = pd.read_csv("/mnt/work1/users/hoffmangroup/zhiyuanl/BEHST/bin/BEHST-results/vista_LIMB_sorted_bed_gProfiler_results_QUERY28000_TARGET18100_revGO_term_list_rand1591227782.behst", 
                   sep='\t', header=0)

# intersect with reference
limb_res = limb.merge(limb_ref, on=['term_id', 'source', 'term_name'], 
                   how='left', indicator=True)
limb_pos = limb_res[limb_res['_merge'] == 'both']
limb_neg = limb_res[limb_res['_merge'] == 'left_only']

In [13]:
limb_pos

,p_value_x,term_id,source,term_name,p_value_y,_merge
0,1.150596e-10,GO:0001501,GO:BP,skeletal system development,1.423548e-19,both
1,2.388447e-09,GO:0048598,GO:BP,embryonic morphogenesis,1.207525e-15,both
2,5.034933e-08,GO:0048568,GO:BP,embryonic organ development,1.529934e-04,both
3,1.895330e-07,GO:0048562,GO:BP,embryonic organ morphogenesis,2.089487e-06,both
4,3.258574e-07,GO:0003002,GO:BP,regionalization,1.752402e-11,both
5,1.151070e-06,GO:0009952,GO:BP,anterior/posterior pattern specification,5.286603e-08,both
6,1.785529e-06,GO:0009790,GO:BP,embryo development,3.135152e-12,both
7,1.796065e-06,GO:0048706,GO:BP,embryonic skeletal system development,2.814358e-09,both
8,2.147059e-06,GO:0000981,GO:MF,"DNA-binding transcription factor activity, RNA polymerase II-specific",1.388936e-15,both
9,2.480959e-06,GO:0007389,GO:BP,pattern specification process,1.859436e-13,both


In [14]:
limb_neg

,p_value_x,term_id,source,term_name,p_value_y,_merge
47,0.018272,GO:0002095,GO:CC,caveolar macromolecular signaling complex,NaN,left_only
59,0.049652,GO:1902379,GO:MF,chemoattractant activity involved in axon guidance,NaN,left_only


### Comparison with total shuffling and tss shuffling
#### limb total

In [15]:
limb_total = pd.read_csv("/mnt/work1/users/hoffmangroup/zhiyuanl/BEHST/results/vista_LIMB_SHUFFLED/gprofiler2_output/vista_LIMB_SHUFFLED_bed_QUERY28000_TARGET18100_gprofiler_output", 
                   sep='\t', header=0)

limb_total = limb_total[['p_value', 'term_id', 'source', 'term_name']]
limb_total = limb_total[limb_total['p_value'] <= 0.05]
len(limb_total)

30

In [16]:
# intersect with reference
limb_total_res = limb_total.merge(limb_ref, on=['term_id', 'source', 'term_name'], 
                   how='left', indicator=True)
limb_total_pos = limb_total_res[limb_total_res['_merge'] == 'both']
limb_total_neg = limb_total_res[limb_total_res['_merge'] == 'left_only']

In [17]:
limb_total_pos

,p_value_x,term_id,source,term_name,p_value_y,_merge
10,0.002825,GO:0009888,GO:BP,tissue development,5.856012e-11,both


In [18]:
limb_total_neg

,p_value_x,term_id,source,term_name,p_value_y,_merge
0,6.026773e-08,GO:0070268,GO:BP,cornification,NaN,left_only
1,1.333986e-06,GO:0031424,GO:BP,keratinization,NaN,left_only
2,2.874554e-06,GO:0030216,GO:BP,keratinocyte differentiation,NaN,left_only
3,5.313839e-05,GO:0009913,GO:BP,epidermal cell differentiation,NaN,left_only
4,3.998853e-04,GO:0010896,GO:BP,regulation of triglyceride catabolic process,NaN,left_only
5,5.903601e-04,GO:0043588,GO:BP,skin development,NaN,left_only
6,6.546098e-04,GO:0090207,GO:BP,regulation of triglyceride metabolic process,NaN,left_only
7,8.311337e-04,GO:0008544,GO:BP,epidermis development,NaN,left_only
8,1.687502e-03,GO:0030855,GO:BP,epithelial cell differentiation,NaN,left_only
9,2.042889e-03,GO:0010898,GO:BP,positive regulation of triglyceride catabolic process,NaN,left_only


#### limb tss

In [19]:
limb_tss = pd.read_csv("/mnt/work1/users/hoffmangroup/zhiyuanl/BEHST/results/randomlyShuffledChromWide_vista_LIMB_sorted/gprofiler2_output/randomlyShuffledChromWide_vista_LIMB_sorted_bed_QUERY28000_TARGET18100_gprofiler_output", 
                   sep='\t', header=0)

limb_tss = limb_tss[['p_value', 'term_id', 'source', 'term_name']]
limb_tss = limb_tss[limb_tss['p_value'] <= 0.05]
len(limb_tss)

10

In [20]:
# intersect with reference
limb_tss_res = limb_tss.merge(limb_ref, on=['term_id', 'source', 'term_name'], 
                   how='left', indicator=True)
limb_tss_pos = limb_tss_res[limb_tss_res['_merge'] == 'both']
limb_tss_neg = limb_tss_res[limb_tss_res['_merge'] == 'left_only']

In [21]:
limb_tss_pos

,p_value_x,term_id,source,term_name,p_value_y,_merge


In [22]:
limb_tss_neg

,p_value_x,term_id,source,term_name,p_value_y,_merge
0,0.035037,GO:0070098,GO:BP,chemokine-mediated signaling pathway,NaN,left_only
1,0.000026,GO:0016493,GO:MF,C-C chemokine receptor activity,NaN,left_only
2,0.000035,GO:0019957,GO:MF,C-C chemokine binding,NaN,left_only
3,0.000059,GO:0001637,GO:MF,G protein-coupled chemoattractant receptor activity,NaN,left_only
4,0.000059,GO:0004950,GO:MF,chemokine receptor activity,NaN,left_only
5,0.000270,GO:0019956,GO:MF,chemokine binding,NaN,left_only
6,0.032747,GO:0008528,GO:MF,G protein-coupled peptide receptor activity,NaN,left_only
7,0.043529,GO:0001653,GO:MF,peptide receptor activity,NaN,left_only
8,0.049614,GO:0035717,GO:MF,chemokine (C-C motif) ligand 7 binding,NaN,left_only
9,0.049614,GO:0071791,GO:MF,chemokine (C-C motif) ligand 5 binding,NaN,left_only


## Reference GO list from all tissues

In [23]:
all_ave = calculate_mean([limb_1, limb_2, forebrain_1, forebrain_2, midbrain_1, midbrain_2, 
                          hindbrain_1, hindbrain_2, 
                       heart_1, heart_2, liver_1, liver_2, neural_tube_1, neural_tube_2, facial_1, facial_2])

all_ave = all_ave[all_ave.pme_TPM >= 1]
all_ave['gene_id'] = all_ave['gene_id'].str.split('.').str.get(0)

# number of expressed genes
len(all_ave)

18704

In [24]:
all_mapped = pd.merge(all_ave, gene_ids, on='gene_id', how='inner')

# number of mapped genes
len(all_mapped)

14897

In [25]:
# all_mapped[['Gene stable ID']].to_csv('../BEHST/data/06_24_reference_gene_id_all', header=None, index=None)

In [26]:
all_ref = pd.read_csv("../BEHST/results/06-24/go_list_all", sep="\t", header=0)
all_ref = all_ref[all_ref.p_value <= 0.05]
all_ref

,p_value,term_id,source,term_name
0,4.940656e-324,GO:0043231,GO:CC,intracellular membrane-bounded organelle
1,4.940656e-324,GO:0043227,GO:CC,membrane-bounded organelle
2,4.940656e-324,GO:0005622,GO:CC,intracellular
3,4.940656e-324,GO:0005737,GO:CC,cytoplasm
4,4.940656e-324,GO:0043226,GO:CC,organelle
...,...,...,...,...
1913,4.925656e-02,GO:0006904,GO:BP,vesicle docking involved in exocytosis
1914,4.925656e-02,GO:0051985,GO:BP,negative regulation of chromosome segregation
1915,4.925656e-02,GO:0021575,GO:BP,hindbrain morphogenesis
1916,4.925656e-02,GO:0051293,GO:BP,establishment of spindle localization


## GREAT: intersect with significant terms in reference list

#### result downloaded from GREAT web query
Test set:    
vista_LIMB_sorted_EDITED_FOR_GREAT.bed (243 genomic regions)  

Background:  
Whole genome background

Assembly:  
Human: GRCh37 (UCSC hg19, Feb. 2009)          

Associated genomic regions:  
Basal+extension (constitutive 5.0 kb upstream and 1.0 kb downstream, up to 1000.0 kb max extension). Curated regulatory domains are included.  
6 of all 243 genomic regions (2.5%) are not associated with any genes. 

Statistical significance: FDR  
threshold: 0.05  
View: siginificant by both

In [27]:
limb_great = pd.read_csv("/mnt/work1/users/hoffmangroup/zhiyuanl/BEHST/results/06-12/06-12-shown-MultipleOntologies.tsv", sep='\t', header=1)
limb_great = limb_great.reset_index()
#limb_great = limb_great[['index', '# Ontology', ' Term Name ', ' Binom Raw P-Value ', '  Binom FDR Q-Val  ', 
#                         '  Hyper FDR Q-Val  ']]
limb_great = limb_great[['index', '# Ontology', ' Term Name ', ' Binom Rank ',
                         ' Binom Raw P-Value ', ' Hyper Rank ']]
limb_great.columns = ['ontology', 'term_name', 'term_id', 'binom_raw_p_value', 'binom_FDR_q_value',
                      'hyper_FDR_q_value']

In [28]:
great_res = limb_great.merge(limb_ref, on=['term_id'], 
                   how='left', indicator=True)
great_res[great_res['_merge'] == 'both']

,ontology,term_name_x,term_id,binom_raw_p_value,binom_FDR_q_value,hyper_FDR_q_value,p_value,source,term_name_y,_merge
8,GO Biological Process,tube development,GO:0035295,3.963168e-10,3.721132e-07,1.907350e-10,1.142134e-03,GO:BP,tube development,both
11,GO Biological Process,embryonic morphogenesis,GO:0048598,8.090736e-10,5.064415e-07,8.150590e-11,1.207525e-15,GO:BP,embryonic morphogenesis,both
14,GO Biological Process,embryonic organ morphogenesis,GO:0048562,1.067187e-08,4.250962e-06,6.585086e-09,2.089487e-06,GO:BP,embryonic organ morphogenesis,both
15,GO Biological Process,embryonic organ development,GO:0048568,1.181012e-08,4.566001e-06,1.970512e-08,1.529934e-04,GO:BP,embryonic organ development,both
17,GO Biological Process,embryo development,GO:0009790,2.616653e-08,9.051554e-06,4.280943e-10,3.135152e-12,GO:BP,embryo development,both
19,GO Biological Process,tube morphogenesis,GO:0035239,5.610827e-08,1.756055e-05,1.996845e-08,1.439291e-02,GO:BP,tube morphogenesis,both
20,GO Molecular Function,"transcription factor activity, sequence-specific DNA binding",GO:0003700,1.689470e-09,3.563937e-06,3.694096e-08,7.006285e-14,GO:MF,DNA-binding transcription factor activity,both
21,GO Molecular Function,"RNA polymerase II transcription factor activity, sequence-specific DNA binding",GO:0000981,1.177067e-07,1.241511e-04,4.573501e-08,1.388936e-15,GO:MF,"DNA-binding transcription factor activity, RNA polymerase II-specific",both


In [29]:
great_neg = great_res[great_res['_merge'] == 'left_only']
great_neg[['ontology', 'term_name_x', 'term_id', 'binom_raw_p_value','_merge']]

,ontology,term_name_x,term_id,binom_raw_p_value,_merge
0,GO Biological Process,negative regulation of macromolecule biosynthetic process,GO:0010558,1.969000e-11,left_only
1,GO Biological Process,negative regulation of cellular biosynthetic process,GO:0031327,3.650388e-11,left_only
2,GO Biological Process,negative regulation of cellular macromolecule biosynthetic process,GO:2000113,5.260388e-11,left_only
3,GO Biological Process,negative regulation of biosynthetic process,GO:0009890,5.767215e-11,left_only
4,GO Biological Process,negative regulation of nucleic acid-templated transcription,GO:1903507,7.676659e-11,left_only
5,GO Biological Process,negative regulation of RNA biosynthetic process,GO:1902679,1.048514e-10,left_only
6,GO Biological Process,"negative regulation of transcription, DNA-templated",GO:0045892,2.447997e-10,left_only
7,GO Biological Process,negative regulation of nitrogen compound metabolic process,GO:0051172,3.208091e-10,left_only
9,GO Biological Process,negative regulation of gene expression,GO:0010629,4.168727e-10,left_only
10,GO Biological Process,negative regulation of RNA metabolic process,GO:0051253,5.461104e-10,left_only


## GREAT-gprofiler Hybrid test

In [30]:
great_all = pd.read_csv("../BEHST/data/limb_GREAT_res_all", sep='\t', header=0)

In [31]:
great_gene_list = []
for i in range(len(great_all)):
    curr_list = great_all.iloc[i, 9]
    curr_list = curr_list.split(',')
    great_gene_list.extend(curr_list)

In [32]:
great_gene_set = set(great_gene_list)

In [33]:
great_gene_df = pd.DataFrame(great_gene_set)
# great_gene_df.to_csv('../BEHST/data/great_limb_gene_id', header=None, index=None)

In [34]:
great_gprofiler = pd.read_csv("../BEHST/results/06-25/limb_great_go_list", sep='\t', header=0)
great_gprofiler_sig = great_gprofiler[great_gprofiler.p_value <= 0.05]
len(great_gprofiler_sig)

320

In [35]:
great_gprofiler_res = great_gprofiler_sig.merge(limb_ref, on=['term_id', 'source', 'term_name'], 
                   how='left', indicator=True)
great_gprofiler_pos = great_gprofiler_res[great_gprofiler_res['_merge'] == 'both']
great_gprofiler_neg = great_gprofiler_res[great_gprofiler_res['_merge'] == 'left_only']

In [36]:
great_gprofiler_pos.head(30)

,p_value_x,term_id,source,term_name,p_value_y,_merge
0,2.736283e-17,GO:0000981,GO:MF,"DNA-binding transcription factor activity, RNA polymerase II-specific",1.388936e-15,both
1,1.811661e-15,GO:0003700,GO:MF,DNA-binding transcription factor activity,7.006285e-14,both
2,7.498709e-14,GO:0140110,GO:MF,transcription regulator activity,9.129339e-11,both
3,8.925696e-14,GO:0006357,GO:BP,regulation of transcription by RNA polymerase II,1.969506e-09,both
4,7.054216e-12,GO:0006366,GO:BP,transcription by RNA polymerase II,1.246135e-08,both
5,6.080727e-11,GO:0009887,GO:BP,animal organ morphogenesis,6.804185e-12,both
6,4.628911e-10,GO:0003002,GO:BP,regionalization,1.752402e-11,both
7,5.089955e-10,GO:0043565,GO:MF,sequence-specific DNA binding,7.987161e-13,both
8,6.266447e-10,GO:0000790,GO:CC,nuclear chromatin,1.702319e-14,both
9,1.300844e-09,GO:0048598,GO:BP,embryonic morphogenesis,1.207525e-15,both


In [37]:
great_gprofiler_neg

,p_value_x,term_id,source,term_name,p_value_y,_merge
10,2.667935e-09,GO:0021537,GO:BP,telencephalon development,NaN,left_only
12,4.380492e-09,GO:0048699,GO:BP,generation of neurons,NaN,left_only
18,8.161508e-09,GO:0007399,GO:BP,nervous system development,NaN,left_only
20,9.398389e-09,GO:0030900,GO:BP,forebrain development,NaN,left_only
22,1.212050e-08,GO:0007507,GO:BP,heart development,NaN,left_only
...,...,...,...,...,...,...
314,4.505809e-02,GO:0035270,GO:BP,endocrine system development,NaN,left_only
315,4.558823e-02,GO:0072087,GO:BP,renal vesicle development,NaN,left_only
316,4.681536e-02,GO:0061311,GO:BP,cell surface receptor signaling pathway involved in heart development,NaN,left_only
317,4.681536e-02,GO:0003338,GO:BP,metanephros morphogenesis,NaN,left_only
